### To do list:
#### 1. ~~Скачать датасет *VGGSound*. Ссылка: https://huggingface.co/datasets/Loie/VGGSound/tree/main~~
#### 2. ~~Все видео-файлы заменить на случайный кадр из них~~
#### 2.1 ~~Перевести wav файлы к частоте 48кГц~~
#### 3. ~~Реализовать DataLoader wav->ipeg~~
#### 4. ~~Встроить аудио-энкодер. Ссылка: https://github.com/archinetai/archisound~~
#### 5. ~~Реализовать сценарий обучения и обучить модель~~

In [1]:
import torch
from utils.config import ModelConfig
from models.unet import UNetWithCrossAttention
from models.diffusion import Diffusion
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from utils.SoundDataset import SoundDataset
from torch.utils.data import DataLoader

In [2]:
image_path = "data/images"
sound_path = "data/sounds"

data = SoundDataset(image_path, sound_path)
train_data, val_data = torch.utils.data.random_split(data, [197889-10000, 10000])

train_loader = DataLoader(train_data, 
                          batch_size=128,
                          num_workers=8,
                          pin_memory=True,
                          shuffle=True, 
                          drop_last=True)
                         
val_loader = DataLoader(val_data, 
                        batch_size=8,
                        num_workers=8,
                        pin_memory=True,
                        shuffle=False, 
                        drop_last=True)

In [3]:
from archisound import ArchiSound

device = "cuda" if torch.cuda.is_available() else "cpu"

autoencoder = ArchiSound.from_pretrained("dmae1d-ATC64-v2").to(device)

2025-05-19 11:02:06.942983: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 11:02:06.950350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747641726.958814   35632 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747641726.961292   35632 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-19 11:02:06.970762: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
config = ModelConfig({"image_size": 128, "audio_ctx_dim": 32})

# Инициализация
diffusion = Diffusion(timesteps=1000, image_size=128, device=device)
model = UNetWithCrossAttention(config).to(device)

train_losses = []
val_losses = []

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

unconditional_prob = 0.08

epoch = 4
for _ in tqdm(range(epoch)):
    epo_train_losses = []
    epo_val_losses = []
    model.train()
    for audio, images in train_loader:

        if torch.rand(1) < unconditional_prob:
            audio_embeds = None
        else:
            with torch.no_grad():
                audio_embeds = autoencoder.encode(audio.to(device)) # [B, d_audio, seq_len] [64, 32, 431]
                
            audio_embeds = audio_embeds.permute(0, 2, 1)  # [B, seq_len, d_audio]

        images = images.to(device)
        
        optimizer.zero_grad()
        loss = diffusion.loss_fn(model, images, audio_embeds)
        loss.backward()
        optimizer.step()

        epo_train_losses.append(loss.item())
        
    scheduler.step()

    train_losses.append(sum(epo_train_losses)/len(epo_train_losses))
    # валидация
    model.eval()
    for audio, images in val_loader:
        with torch.no_grad():
            audio_embeds = autoencoder.encode(audio.to(device))
            
            audio_embeds = audio_embeds.permute(0, 2, 1)

            # audio_embeds = torch.zeros((8, 431, 32)).to(device)
            images = images.to(device)
            
            loss = diffusion.loss_fn(model, images, audio_embeds)
    
            epo_val_losses.append(loss.item())

    val_losses.append(sum(epo_val_losses)/len(epo_val_losses))
    

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
plt.plot(train_losses, label='train')
plt.plot(val_losses, label='val')
plt.grid()
plt.legend()

In [ ]:
# plt.plot(epo_train_losses, label='train')
plt.plot(epo_train_losses, label='val')
plt.grid()
plt.legend()

In [ ]:
audio,image = next(iter(val_loader))
audio_embeds = autoencoder.encode(audio.to(device))
audio_embeds = audio_embeds.permute(0, 2, 1)

model.eval()
generated_image = diffusion.reverse_process(
    model,
    audio_embeds,
    guidance_scale=7.5,
    batch_size=8,
    use_ddim=True,
    timesteps=100  #число шагов
)

# 16 384
# 32 768

In [ ]:
plt.imshow(torch.permute(generated_image[0].cpu()*0.5+0.5, (1,2,0)).numpy())

In [ ]:
# датасет обходится за 25 мин, 40 сек
# при num_workers = 8 обходится за ~6 мин 30 сек
# 1300 сек на эпоху

# при двух attention блоках средние потери падают до 0.54 на 11 эпохах
# при единственном блоке потери не пробивали отсечку в 0.9

# 3 attention блока дали 0.84 на 4 эпохах. При продолжении обучения к концу 7 эпохи потери не изменились

In [ ]:
# for el in model.mid_block.attn.parameters():
#     print("Attention stats:", el.mean().item(), el.max().item())

In [ ]:
# torch.save(model.state_dict(), "weights2.pth")

In [ ]:
scheduler.get_lr()

In [5]:
a = list(model.parameters())

In [8]:
for i, (name, param) in enumerate(model.named_parameters()):
    print(f"{i} | {name}: {param.shape}")

0 | down_blocks.0.conv1.weight: torch.Size([32, 3, 3, 3])
1 | down_blocks.0.conv1.bias: torch.Size([32])
2 | down_blocks.0.conv2.weight: torch.Size([32, 32, 3, 3])
3 | down_blocks.0.conv2.bias: torch.Size([32])
4 | down_blocks.0.BN1.weight: torch.Size([32])
5 | down_blocks.0.BN1.bias: torch.Size([32])
6 | down_blocks.0.downsample.weight: torch.Size([32, 32, 3, 3])
7 | down_blocks.0.downsample.bias: torch.Size([32])
8 | down_blocks.0.time_embed.0.weight: torch.Size([32, 1])
9 | down_blocks.0.time_embed.0.bias: torch.Size([32])
10 | down_blocks.0.time_embed.2.weight: torch.Size([32, 32])
11 | down_blocks.0.time_embed.2.bias: torch.Size([32])
12 | down_blocks.1.conv1.weight: torch.Size([64, 32, 3, 3])
13 | down_blocks.1.conv1.bias: torch.Size([64])
14 | down_blocks.1.conv2.weight: torch.Size([64, 64, 3, 3])
15 | down_blocks.1.conv2.bias: torch.Size([64])
16 | down_blocks.1.BN1.weight: torch.Size([64])
17 | down_blocks.1.BN1.bias: torch.Size([64])
18 | down_blocks.1.downsample.weight: torc

In [2]:
from transformers import Wav2Vec2Model

model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self", 
                                      torch_dtype=torch.float16, 
                                      attn_implementation="flash_attention_2").cuda()


2025-05-22 17:53:24.233455: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 17:53:24.241455: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747925604.250008  108046 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747925604.252504  108046 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747925604.259660  108046 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [5]:
sample = torch.rand((2,10000)).half().cuda()

print(sample.shape)



torch.Size([2, 10000])


In [8]:
obj = model(sample)

In [12]:
obj[0].shape

torch.Size([2, 31, 1024])

In [13]:
obj[1].shape

torch.Size([2, 31, 512])

In [18]:
import torch
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2FeatureExtractor

# # Предположим, у вас есть стерео тензор audio_tensor формы (2, num_samples)
# def stereo_to_mono(audio_tensor):
#     # Усредняем оба канала (можно также выбрать один канал)
#     return torch.mean(audio_tensor, dim=0, keepdim=True)

# def preprocess_audio(audio_mono, target_sample_rate=16000):
#     # Если частота дискретизации не 16 кГц, ресемплируем
#     if target_sample_rate != 16000:
#         resampler = torchaudio.transforms.Resample(
#             orig_freq=target_sample_rate,
#             new_freq=16000
#         )
#         audio_mono = resampler(audio_mono)
#     return audio_mono

# Загружаем модель и feature extractor (можно заменить на другую версию)
model_name = "facebook/wav2vec2-base-960h"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

def get_wav2vec2_features(audio_stereo_tensor, original_sample_rate):
    # 1. Конвертируем стерео в моно
    # audio_mono = stereo_to_mono(audio_stereo_tensor)
    
    # # 2. Ресемплируем до 16 кГц если нужно
    # audio_preprocessed = preprocess_audio(audio_mono, original_sample_rate)
    
    # 3. Нормализуем и извлекаем features
    inputs = feature_extractor(
        audio_stereo_tensor.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )
    
    # 4. Получаем скрытые состояния
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 5. Возвращаем усреднённые features по времени (можно использовать и другие агрегации)
    last_hidden_state = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
    return torch.mean(last_hidden_state, dim=1)    # (batch_size, hidden_size)

# Пример использования:
# Предположим, у нас есть стерео тензор (2 канала, 44100 Гц)
stereo_audio = torch.randn(1, 44100 * 100)  # 3 секунды аудио
sample_rate = 44100

features = get_wav2vec2_features(stereo_audio, sample_rate)
print(f"Размерность признакового вектора: {features.shape}")  # Ожидаем [1, 768] для wav2vec2-base

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Размерность признакового вектора: torch.Size([1, 768])
